In [ ]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob
import platform
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Paths Input Here
analysis = "roi_roi_overlap/vta_and_bf_ch_nuclei"
if platform.uname().system == 'Darwin': #------------------------------Mac OS X---------------------------------------------------------------
    conn_path = r'/Users/cu135/Dropbox (Partners HealthCare)/memory/seeds/non_fl_vtas_bilateral'
    roi_path = r'/Users/cu135/Dropbox (Partners HealthCare)/neurotransmitters/basal_forebrain/generated_spherical_rois/cholinergic_nuclei_maxima'
    
    # clin_path = 'path to clinical values'
    out_dir = os.path.join(os.path.dirname(conn_path), f'{analysis}')
    
    #roi_names = '<path to roi name location>'
    print('I have set pathnames in the Mac style')
    print('I will save to:', out_dir)
else: #----------------------------------------------------------------Windows----------------------------------------------------------------
    print('add the paths')

if os.path.isdir(out_dir) != True:
    os.makedirs(out_dir)

save = True

In [ ]:
#Get and Prepare Dataframes
from calvin_utils.import_matrices import import_matrices_from_folder
df_one = import_matrices_from_folder(conn_path, file_pattern='/*.nii')
display(df_one)

In [ ]:
roi_df = import_matrices_from_folder(roi_path, file_pattern='/*.nii')
display(roi_df)

In [ ]:
# Calculate overlap for each ROI and column
overlap = {}
#Set specific ROI to 1 if nonzero
roi_df[roi_df!= 0] = 1
#Set specific nifti of interest to 1 if nonzero
df_one[df_one!= 0] = 1

#Iterate over ROI niftis
for i in range(0, len(roi_df.columns)):
    #Initialize
    row = []
    
    #Iterate over nifti of comparison
    for j in range(len(df_one.columns)):
        # Find all values where the data frames overlap (both == 1)
        numerator = np.sum(np.logical_and(roi_df.iloc[:, i], df_one.iloc[:, j]))

        # Find total number of nonzero values in df_one
        denominator = np.sum(roi_df.iloc[:, i])

        # Find total percent of overlap
        overlap_percent = numerator / denominator * 100
        row.append(overlap_percent)
    overlap[roi_df.columns[i]] = row


In [ ]:
# Create a data frame with the overlap values
overlap_df = pd.DataFrame(overlap, index=df_one.columns)

# overlap_df = pd.DataFrame(overlap, columns=df_one.columns, ).transpose()
display(overlap_df)

#Merge the dataframe back to the orignal
if save:
    overlap_df.to_csv(os.path.join(out_dir, 'overlap_df.csv'))
    print('saved to ', out_dir)